In [1]:
# import library
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
# import dataset
df = pd.read_csv('./input.txt', delimiter = "\t", header=None)
df.columns = ['Raw']

In [3]:
# create new dataset
# Grant Period
new_dataset = pd.DataFrame(data = np.array(df.iloc[df[df['Raw'].str.match('Grant period')].index+1]), columns = ['Grant period'])

# Appropriated budget
new_dataset['Appropriated budget'] = np.array(df.iloc[df[df['Raw'].str.match('Appropriated budget')].index+1])

# Executing entity
new_dataset['Executing entity'] = np.array(df.iloc[df[df['Raw'].str.match('Executing entity')].index+1])

# Reference number
new_dataset['Reference number'] = np.array(df.iloc[df[df['Raw'].str.match('Reference number')].index+1])

# Project
new_dataset['Project'] = np.array(df[~df['Raw'].isin(new_dataset['Grant period'])][~df['Raw'].isin(new_dataset['Appropriated budget'])][~df['Raw'].isin(new_dataset['Executing entity'])][~df['Raw'].isin(new_dataset['Reference number'])][~df['Raw'].str.contains("Grant period")][~df['Raw'].str.contains("Appropriated budget")][~df['Raw'].str.contains("Executing entity")][~df['Raw'].str.contains("Reference number")]);


In [4]:
# Create Desired Format

# Date
dataset = pd.DataFrame(new_dataset['Grant period'].str.split(' ',1).tolist(),
                                 columns = ['Start Date','End Date'])

# Budget
dataset['Budget'] = new_dataset['Appropriated budget'].str.replace(',', '').str.replace('€', '').astype(float)

# Institution
dataset['Institution'] = new_dataset['Executing entity']

# Reference Number
dataset['FKZ'] = new_dataset['Reference number']

# Project Type
dataset['Project type'] = new_dataset['Project'].apply(lambda x: "Joint Project" if "jointproject" in x.lower().strip().replace(" ", "") else "")

# Project Type Binary
dataset['Project type binary'] = new_dataset['Project'].apply(lambda x: 1 if "jointproject" in x.lower().strip().replace(" ", "") else 0)

# Project Name
dataset['Project'] = new_dataset['Project'].str.replace('Joint projects', '').str.replace('Joint project', '').str.replace('[:,@]', '').str.strip()

In [5]:
# data sources link
excel_to_dir = './excel_data.csv'

# read data from excel file
# please make sure that xlrd package is installed
# if xlrd > 2.0.1 then also install openpyxl package
dataset.to_csv(excel_to_dir)

In [6]:
dataset.head()

,Start Date,End Date,Budget,Institution,FKZ,Project type,Project type binary,Project
0,2015-06-01,2017-02-28,61135.25,"Holzapfel Metallveredelung GmbH, Sinn",0325524D,Joint Project,1,Development work on alkaline pressure electrol...
1,2012-11-01,2016-12-31,250250.00,"Power-to-Gas Invest GmbH, Stuttgart",0325524B,Joint Project,1,'Development work on alkaline pressure electro...
2,2012-11-01,2013-09-30,33103.95,"ENERTRAG HyTec GmbH - project development, Sch...",0325524C,Joint Project,1,Development work on alkaline pressure electrol...
3,2012-11-01,2017-02-28,2724498.68,Center for Solar Energy and Hydrogen Research ...,0325524A,Joint Project,1,'Development work on alkaline pressure electro...
4,2012-07-01,2015-06-30,279261.00,"FUMATECH BWT GmbH, Bietigheim-Bissingen",03EK3012B,Joint Project,1,MaPEI Membrane electrode units for alkaline PE...
